In [ ]:
import boto3
from datetime import datetime

# Print execution info
print(f"Execution started at: {datetime.now()}")

# Get current IAM role
sts = boto3.client('sts')
identity = sts.get_caller_identity()
print(f"Current Role ARN: {identity['Arn']}")
print(f"Account ID: {identity['Account']}")

In [ ]:
# Parameters cell - will be replaced by Papermill
agent_name = "calculator_agent"
agent_llm = "us.anthropic.claude-3-5-sonnet-20241022-v2:0"
force_recreate = True
kb_name = "mortgage-kb"

In [ ]:
import boto3
import time
from sagemaker_studio import Project

print("🚀 Starting Comprehensive Bedrock Agent Workflow")

# Get project connections dynamically (account-agnostic)
proj = Project()
region = boto3.Session().region_name
iam_conn = proj.connection('default.iam')
role = iam_conn.iam_role
s3_shared_conn = proj.connection('default.s3_shared')
bucket = s3_shared_conn.data.s3_uri.rstrip('/').split('/')[-2]

print(f"✅ Region: {region}")
print(f"✅ IAM Role: {role}")
print(f"✅ S3 Bucket: {bucket}")
print(f"✅ LLM: {agent_llm}")

In [ ]:
# Download utils module from S3
import subprocess
import sys
import os

print("\n📦 Downloading utils module from S3...")
s3_path = f's3://{bucket}/shared/genai/bundle/agent-code/utils/'
print(f"   S3 Path: {s3_path}")

result = subprocess.run([
    'aws', 's3', 'sync',
    s3_path,
    './utils/',
    '--region', region
], capture_output=True, text=True)

if result.returncode != 0:
    print(f"❌ S3 sync failed: {result.stderr}")
    raise Exception(f"Failed to download utils: {result.stderr}")

# Verify download
if os.path.exists('./utils'):
    files = os.listdir('./utils')
    print(f"✅ Utils module downloaded: {len(files)} files")
    for f in files:
        print(f"   - {f}")
else:
    print("❌ Utils directory not found!")
    raise Exception("Utils directory was not created")

# Add current directory to Python path
if '.' not in sys.path:
    sys.path.insert(0, '.')
    print("✅ Added current directory to sys.path")

In [ ]:
# Early cleanup from previous runs (ignore errors)
print("\n🧹 Early cleanup from previous runs...")

try:
    from utils.bedrock_agent import Agent
    from utils.knowledge_base_helper import KnowledgeBaseHelper
    
    kb_helper = KnowledgeBaseHelper()
    
    # Try to delete specific resources that might conflict
    if Agent.exists("mortgage_test_agent"):
        Agent.delete_by_name("mortgage_test_agent")
        print("   Deleted mortgage_test_agent")
    
    kb_helper.delete_kb(kb_name, delete_s3_bucket=True, delete_iam_roles_and_policies=True)
    print(f"   Deleted knowledge base: {kb_name}")
    
    print("✅ Early cleanup complete")
except Exception as e:
    print(f"⚠️ Early cleanup warning (resources may not exist): {e}")

In [ ]:
import os
print(f"IMAGE_VERSION: {os.environ.get('IMAGE_VERSION', 'Not set')}")
print(f"SAGEMAKER_INTERNAL_IMAGE_URI: {os.environ.get('SAGEMAKER_INTERNAL_IMAGE_URI', 'Not set')}")

In [ ]:
from datetime import datetime
from zoneinfo import ZoneInfo
print(f"Execution Start Time (EST): {datetime.now(ZoneInfo('America/New_York')).strftime('%Y-%m-%d %H:%M:%S %Z')}")
print("version 1.0.0")

In [ ]:
# Package upgrade skipped - using environment default
print('Using default sagemaker_studio package from environment')

In [ ]:
# Cleanup from previous runs
print("\n🧹 Cleaning up from previous runs...")

from utils.bedrock_agent import Agent
import boto3

# Delete agents from previous runs
agent_names = ["poet-agent", "calculator_agent", "test_agent", "test_user_data_agent", "mortgage_test_agent"]
for name in agent_names:
    try:
        if Agent.exists(name):
            Agent.delete_by_name(name)
            print(f"   Deleted agent: {name}")
    except Exception as e:
        print(f"   Could not delete {name}: {e}")

# Delete Lambda functions from previous runs
lambda_client = boto3.client('lambda', region_name=region)
lambda_names = ["mask_string", "mask_string_external", "transmorgifier", "get_user_data"]
for name in lambda_names:
    try:
        lambda_client.delete_function(FunctionName=name)
        print(f"   Deleted Lambda: {name}")
    except lambda_client.exceptions.ResourceNotFoundException:
        pass  # Function doesn't exist, that's fine
    except Exception as e:
        print(f"   Could not delete Lambda {name}: {e}")

print("✅ Cleanup complete")

## Scenario 1: Basic Agent Operations
Create, test existence, invoke, and delete a simple agent

In [ ]:
print("\n📝 Scenario 1: Basic Agent Operations")

Agent.set_force_recreate_default(force_recreate)

# Create simple poet agent
agent = Agent.create(
    "poet-agent",
    "You are a poet. You write short poems about the topics users provide.",
    llm=agent_llm
)

print("✅ Created poet-agent")

# Test existence
exists = Agent.exists("poet-agent")
print(f"✅ Agent exists: {exists}")

# Invoke agent
response = agent.invoke("Write a haiku about clouds")
print(f"✅ Agent response: {response}")

# Delete agent
Agent.delete_by_name("poet-agent")
print("✅ Deleted poet-agent")

## Scenario 2: Agent from YAML Template
Create an agent using a YAML configuration file

In [ ]:
print("\n📄 Scenario 2: Agent from YAML Template")

# Download YAML template from S3
import subprocess
yaml_path = './test_agent.yaml'
subprocess.run([
    'aws', 's3', 'cp',
    f's3://{bucket}/shared/genai/bundle/agent-code/test_agent.yaml',
    yaml_path,
    '--region', region
], check=True)

# Create agent from YAML (name, yaml_file)
test_agent = Agent.create_from_yaml('test_agent', yaml_path)
print("✅ Created test_agent from YAML template")

# Test agent
response = test_agent.invoke("Hello, how are you?")
print(f"✅ Agent response: {response[:100]}...")

## Scenario 3: Tools from Python Functions
Create and attach tools defined as Python functions

In [ ]:
print("\n🔧 Scenario 3: Tools from Python Functions")

# Define mask_string function
def mask_string(input_string: str) -> str:
    """Masks a string by replacing all characters with asterisks.
    
    Args:
        input_string: The string to mask
    
    Returns:
        A string of asterisks with the same length as input
    """
    return '*' * len(input_string)

# Attach tool to test_agent
test_agent.attach_tool_from_function(mask_string)
test_agent.prepare()

print("✅ Attached mask_string tool to test_agent")

# Test the tool
response = test_agent.invoke("Please mask the string 'secret123'")
print(f"✅ Tool response: {response}")

## Scenario 4: Tools via ParameterSchema
Create tools using ParameterSchema for external Lambda functions

In [ ]:
print("\n⚙️  Scenario 4: Tools via ParameterSchema")

from utils.bedrock_agent import Tool, ParameterSchema, ParamType

# Create parameter schema
schema = ParameterSchema.create_with_values(
    name="input_string",
    parameter_type=ParamType.STRING,
    description="The string to mask",
    required=True
)

# Create tool with schema and code file
mask_tool = Tool.create(
    "mask_string_external",
    schema=schema,
    code_file="lambda_mask_string.py",
    description="Masks a string by replacing characters with asterisks"
)

# Attach to agent
test_agent.attach_tool(mask_tool)
test_agent.prepare()

print("✅ Created and attached tool via ParameterSchema")

## Scenario 5: Tools Defined at Agent Creation
Define tools when creating the agent

In [ ]:
print("\n🛠️  Scenario 5: Tools Defined at Agent Creation")

# Define user data function
def get_user_data(user_id: str) -> dict:
    """Gets user data from a database.
    
    Args:
        user_id: The user ID to look up
    
    Returns:
        Dictionary with user information
    """
    return {"user_id": user_id, "name": "Test User", "email": "test@example.com"}

# Create agent first, then attach tool from function
test_user_data_agent = Agent.create(
    name="test_user_data_agent",
    role="You help users look up information",
    llm=agent_llm
)

# Attach tool from Python function
test_user_data_agent.attach_tool_from_function(get_user_data)
test_user_data_agent.prepare()

print("✅ Created agent and attached tool from function")

# Test the agent
response = test_user_data_agent.invoke("Get data for user ID 12345")
print(f"✅ Agent response: {response}")

# Cleanup
test_user_data_agent.delete()
print("✅ Deleted test_user_data_agent")

## Scenario 6: Knowledge Base Integration
Create a knowledge base, upload documents, and attach it to an agent

In [ ]:
# Install opensearch-py for Knowledge Base scenario
%pip install -q opensearch-py

In [ ]:
print("\n📚 Scenario 6: Knowledge Base Integration")

from utils.knowledge_base_helper import KnowledgeBasesForAmazonBedrock
import os

# Initialize Knowledge Base helper
kb_helper = KnowledgeBasesForAmazonBedrock()

# Create sample mortgage documents
print("\n📝 Creating sample mortgage documents...")
os.makedirs('./mortgage_docs', exist_ok=True)

# Sample document 1: Mortgage basics
with open('./mortgage_docs/mortgage_basics.txt', 'w') as f:
    f.write("""
Mortgage Basics

A mortgage is a loan used to purchase real estate. The property serves as collateral for the loan.

Key Terms:
- Principal: The amount borrowed
- Interest Rate: The cost of borrowing, expressed as a percentage
- Term: The length of time to repay the loan (typically 15 or 30 years)
- Down Payment: The upfront payment, usually 3-20% of the purchase price
- PMI: Private Mortgage Insurance, required if down payment is less than 20%

Types of Mortgages:
1. Fixed-Rate Mortgage: Interest rate stays the same for the entire loan term
2. Adjustable-Rate Mortgage (ARM): Interest rate can change periodically
3. FHA Loan: Government-backed loan with lower down payment requirements
4. VA Loan: Available to veterans with no down payment required
""")

# Sample document 2: Mortgage rates
with open('./mortgage_docs/current_rates.txt', 'w') as f:
    f.write("""
Current Mortgage Rates (Example Data)

30-Year Fixed Rate: 6.5% - 7.0%
15-Year Fixed Rate: 5.8% - 6.3%
5/1 ARM: 5.5% - 6.0%

Factors Affecting Your Rate:
- Credit Score: Higher scores get better rates
- Down Payment: Larger down payments reduce risk and rates
- Loan Amount: Jumbo loans may have different rates
- Property Type: Investment properties typically have higher rates
- Debt-to-Income Ratio: Lower ratios are preferred
""")

# Sample document 3: Application process
with open('./mortgage_docs/application_process.txt', 'w') as f:
    f.write("""
Mortgage Application Process

Step 1: Pre-Qualification
Get an estimate of how much you can borrow based on basic financial information.

Step 2: Pre-Approval
Lender verifies your financial information and provides a conditional commitment.

Step 3: House Hunting
Search for properties within your budget.

Step 4: Make an Offer
Submit an offer on a property you want to purchase.

Step 5: Home Inspection and Appraisal
Property is inspected and appraised to determine its value.

Step 6: Underwriting
Lender reviews all documentation and makes final approval decision.

Step 7: Closing
Sign final documents and receive keys to your new home.

Required Documents:
- Pay stubs (last 2 months)
- W-2 forms (last 2 years)
- Tax returns (last 2 years)
- Bank statements (last 2-3 months)
- Photo ID
""")

print("✅ Created 3 sample mortgage documents")

In [ ]:
# Upload documents to S3
print("\n📤 Uploading documents to S3...")

# Create or retrieve knowledge base (this also creates S3 bucket)
kb_id, ds_id = kb_helper.create_or_retrieve_knowledge_base(
    kb_name=kb_name,
    kb_description="Knowledge base containing mortgage information and guidelines",
    embedding_model="amazon.titan-embed-text-v2:0"
)

print(f"✅ Knowledge Base ID: {kb_id}")
print(f"✅ Data Source ID: {ds_id}")

# Get the S3 bucket name
data_bucket = kb_helper.get_data_bucket_name()
print(f"✅ S3 Bucket: {data_bucket}")

# Upload documents to S3
import subprocess
result = subprocess.run([
    'aws', 's3', 'sync',
    './mortgage_docs/',
    f's3://{data_bucket}/',
    '--region', region
], capture_output=True, text=True)

if result.returncode == 0:
    print("✅ Documents uploaded to S3")
else:
    print(f"❌ Upload failed: {result.stderr}")
    raise Exception(f"Failed to upload documents: {result.stderr}")

In [ ]:
# Synchronize/ingest data into knowledge base
print("\n🔄 Starting data ingestion (this may take 2-3 minutes)...")

kb_helper.synchronize_data(kb_id, ds_id)

print("✅ Data ingestion complete")

In [ ]:
# Create agent with knowledge base
print("\n🤖 Creating mortgage advisor agent with knowledge base...")

mortgage_agent = Agent.create(
    name="mortgage_test_agent",
    role="You are a helpful mortgage advisor. Use the knowledge base to answer questions about mortgages, rates, and the application process.",
    kb_id=kb_id,
    kb_descr="Mortgage information including basics, current rates, and application process",
    llm=agent_llm
)

print("✅ Created mortgage advisor agent with knowledge base")

In [ ]:
# Test the agent with knowledge base queries
print("\n💬 Testing agent with knowledge base queries...\n")

# Query 1: Basic mortgage information
print("Query 1: What is a mortgage and what are the main types?")
response1 = mortgage_agent.invoke("What is a mortgage and what are the main types of mortgages available?")
print(f"Response: {response1}\n")
print("-" * 80)

# Query 2: Current rates
print("\nQuery 2: What are the current mortgage rates?")
response2 = mortgage_agent.invoke("What are the current mortgage rates for 30-year and 15-year fixed mortgages?")
print(f"Response: {response2}\n")
print("-" * 80)

# Query 3: Application process
print("\nQuery 3: What documents do I need for a mortgage application?")
response3 = mortgage_agent.invoke("What documents do I need to provide when applying for a mortgage?")
print(f"Response: {response3}\n")
print("-" * 80)

print("\n✅ Knowledge base integration test complete!")

In [ ]:
# Cleanup: Delete the agent (knowledge base will be cleaned up in final cleanup cell)
print("\n🧹 Cleaning up mortgage agent...")
mortgage_agent.delete()
print("✅ Mortgage agent deleted")